In [6]:
import pandas as pd
import datetime as dt

# Загрузка данных 
akp = pd.read_excel('akp_mat_2022.xlsx')
vmt = pd.read_excel('v_mat_2022.xlsx')
akp_od = pd.read_excel('akp_od_2022-2.xlsx')
v_od = pd.read_excel('v_od_2022.xlsx')
ha = pd.read_excel('ha_2022.xlsx', sheet_name='NV')


# Функция для преобразования времени присадки в секунды
def to_time(time_str):
    hh, mm, ss = time_str.split(':')
    return int(hh) * 3600 + int(mm) * 60 + int(round(float(ss), 2))


In [7]:
akp.shape

(77396, 8)

In [8]:
akp.head(10)

,ID,Номер обработки,Дата и время отдачи,ID_M,Название материала,Название рецепта,Источник,"Вес материала, кг."
0,1RZ668,1,"2021-12-31 23:15:41,909",140107,Известь,SlagForm,Bin,198.000
1,1RZ668,1,"2021-12-31 23:15:41,909",205065,АТФ-75,SlagForm,Bin,73.000
2,1RZ668,1,"2021-12-31 23:18:05,837",140107,Известь,AlloyFinal,Bin,296.000
3,1RZ668,1,"2021-12-31 23:18:05,837",205065,АТФ-75,AlloyFinal,Bin,72.000
4,1RZ668,1,"2021-12-31 23:34:57,846",202107,ФС75,AlloyInitial,Bin,189.000
5,1RZ668,1,"2021-12-31 23:34:57,846",140107,Известь,AlloyInitial,Bin,148.000
6,1RZ668,1,"2021-12-31 23:34:57,846",203130,УСМ 3-15 мм,AlloyInitial,Bin,126.000
7,1RZ668,1,"2021-12-31 23:34:57,846",202007,MnSi17А,AlloyInitial,Bin,153.000
8,1RZ668,1,"2021-12-31 23:54:16,169",323141,катанка алюмин АКЛП,Trimming1,Wire,32.000
9,1RZ668,1,"2021-12-31 23:56:33,396",202049,ПП-СК30 проволока,Trimming2,Wire,64.516


In [10]:
akp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77396 entries, 0 to 77395
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   77396 non-null  object 
 1   Номер обработки      77396 non-null  int64  
 2   Дата и время отдачи  77396 non-null  object 
 3   ID_M                 77396 non-null  int64  
 4   Название материала   77396 non-null  object 
 5   Название рецепта     77396 non-null  object 
 6   Источник             77396 non-null  object 
 7   Вес материала, кг.   77396 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 4.7+ MB


In [ ]:
# Фоматироавние данных по присадкам 
akp = akp.loc[akp['Название материала'].isin(['катанка алюмин АКЛП', 'ПП-СК30 проволока']),
                ['ID', 'Дата и время отдачи', 'Название материала']].rename(columns={'Дата и время отдачи':'date_time', 'Название материала':'name_materials'})

akp['date_time'] = pd.to_datetime(akp['date_time'])
akp['date_time'] = akp['date_time'].dt.time
akp['date_time'] = akp['date_time'].apply(lambda x: str(x))
akp['date_time'] = akp['date_time'].apply(to_time)

akp = akp.pivot_table(index='ID', columns='name_materials', values='date_time', aggfunc=['max', 'min'])[['max', 'min']].reset_index().iloc[:, [0, 2, 3]]
akp.columns = ['ID', 'aluminium_max_time', 'calcium_min_time']

# Часть данных по присадкам кальция находится в файле v_mat, поэтму находим присадки алюминия в akp_mat без присадки кальция 
akp_only_aluminium = akp[akp['calcium_min_time'].isnull()][["ID", "aluminium_max_time"]]
# Удаляем строки с пустым значениями